In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import h5py
import logging
from os import path
from pandas import HDFStore
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import pathlib
import shutil
import tempfile
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import timeseries_dataset_from_array


import math

from utils import load_residues_dataset, split_train_test, windowed_dataset, split_train_test_val
from config import *

%matplotlib inline

window_size=round(4096/4)

HDFStore=pd.HDFStore
hdf = HDFStore(h5filename)
h5filename=h5filename
print(h5filename)
hdf.keys()

/data/elekin/data/results/handwriting/archimedean-17.h5


['/source/dataset',
 '/source/labels',
 '/results/residues/features',
 '/results/residues/rd',
 '/results/radius/features',
 '/results/radius/r']

In [2]:
residues=hdf['/results/residues/rd']
labels=hdf['/source/labels']

In [3]:
hdf.close()

In [7]:
rolled =residues.rolling(round(4096/16)).std().dropna()
rolled.describe()

,C01_11,C01_12,C02_11,C02_12,C03_11,C03_12,C04_11,C04_12,C05_11,C05_12,...,control2_01,control2_02,control3_01,control3_02,control4_01,control4_02,control6_01,control6_02,control7_01,control7_02
count,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,...,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000,3841.000000
mean,29.608315,24.635253,29.937969,37.304354,22.586074,30.466801,33.041917,30.190067,38.624101,27.533153,...,59.642004,31.082821,64.339435,41.205316,99.378109,41.428295,53.405596,38.037733,40.292484,36.764331
std,10.319607,13.461991,14.349422,27.618238,8.450348,12.839783,14.240844,13.321235,16.995977,11.984692,...,48.980261,17.824924,74.473012,42.341037,104.077622,19.682270,54.791133,13.718908,28.194947,28.651363
min,10.465534,3.768582,7.828024,4.146649,6.015019,11.547645,7.665631,11.954557,8.443727,6.249616,...,12.247924,7.916369,11.581669,11.966519,20.971488,9.040138,12.812669,11.670371,11.576587,9.365653
25%,22.565670,15.123158,19.309250,16.097841,15.683448,20.843118,23.109047,18.031392,24.798825,18.709808,...,32.875997,18.972532,34.008533,23.128025,52.022034,24.981298,23.224120,29.212238,23.301624,22.219017
50%,27.413607,22.129791,26.309766,25.797659,22.756174,29.334299,32.367934,27.506147,37.733462,26.168477,...,47.727315,29.322149,49.708925,31.735030,73.901908,39.825411,38.909549,38.428227,33.536709,29.230624
75%,37.076994,31.842144,37.404580,53.354334,28.569266,35.338651,41.385368,37.609440,49.933597,36.194390,...,65.736838,37.287423,63.184969,46.008286,94.935978,55.150159,54.394438,47.324724,46.853046,40.857373
max,63.479527,74.167295,88.395436,115.503629,61.589324,71.039395,72.117287,64.583928,86.233774,63.914891,...,264.233796,332.963327,459.148015,604.253136,530.069768,254.265556,261.680376,309.459442,322.124846,459.235073


In [6]:
def time_serie_windowing(ts, label, window_size):
    ds = tf.data.Dataset.from_tensor_slices(tf.constant(ts))
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size))
    ds = ds.map(lambda window: (window, [label]))
    return ds

Window function is applied to Dataset objects. Same windows lenghts for each time series corresponding to each subject are built asigning the rigth label. Then they are included in same Dataset object.

In [7]:
#windowing parameters
x = rolled.T.values.astype('float32')
y=labels.T.values.astype('int8')
shuffle_buffer = num_features

#series, window_size, batch_size,label

for i,subject in enumerate(zip(x,y)):
    new = time_serie_windowing(subject[0], subject[1], window_size)
    if i>0:
        dataset = tf.data.Dataset.concatenate(dataset,new)
    else:
        print("new")
        dataset = new
dataset.cache()
    #print(fetaures, label)

dataset.element_spec

new


(TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 TensorSpec(shape=(1,), dtype=tf.int8, name=None))

In [8]:
#list(dataset.as_numpy_iterator())[0]
for features, label in new.take(2):
    print(features.shape, label)

(1024,) tf.Tensor([1], shape=(1,), dtype=int8)
(1024,) tf.Tensor([1], shape=(1,), dtype=int8)


In [14]:
for idx, reg in enumerate(new):
    pass
print(idx)

2817


In [10]:
tf.data.experimental.cardinality(dataset).numpy()

-2

In [12]:
DATASET_SIZE=idx
idx

140899

In [13]:
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
batch_size = 8

full_dataset = dataset
full_dataset = full_dataset.shuffle(DATASET_SIZE)
train_dataset = full_dataset.take(train_size).batch(batch_size).cache()
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size).batch(batch_size).cache()
test_dataset = test_dataset.take(test_size).batch(batch_size).cache()

In [13]:
train_dataset.element_spec

(TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1), dtype=tf.int8, name=None))

In [16]:
for ts, label in train_dataset.take(1):
    print(ts)

KeyboardInterrupt: 

## FCNN SIMPLE ARCHITECTURE ANALISYS

In [18]:
# # #Early stop configuration
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=1e-3,
  patience=200)

training_earlystop_callback = EarlyStopping(
  monitor='accuracy', min_delta=1e-4,
  patience=200)

def get_callbacks(name):
    return [
        tfdocs.modeling.EpochDots(),
        earlystop_callback,
        #tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, min_delta=1e-5),
        #tf.keras.callbacks.TensorBoard(logdir/name),
      ]

def compile_and_fit(model, train_dataset, test_dataset, name, optimizer=None, max_epochs=1000):
    tf.keras.backend.clear_session()# avoid clutter from old models and layers, especially when memory is limited
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    tf.random.set_seed(seed) # establecemos la semilla para tensorflow
    history = model.fit(train_dataset, 
                        use_multiprocessing=True, 
                        validation_data=test_dataset, 
                        epochs=max_epochs, 
                        callbacks=get_callbacks(name),
                        verbose=1, 
                        shuffle=True)
    return history

# Many models train better if you gradually reduce the learning rate during training. 
# Use optimizers.schedules to reduce the learning rate over time:
def get_optimizer(steps_per_epoch=1, lr=1e-4, multiplier=1000):
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(lr,
                                                                 decay_steps=steps_per_epoch*multiplier,
                                                                 decay_rate=1,
                                                                 staircase=False)
    return tf.keras.optimizers.Adam(lr_schedule)

Many models train better if you gradually reduce the learning rate during training. 
Use optimizers.schedules to reduce the learning rate over time:

In [15]:
size_histories = {}
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
tiny = tf.keras.models.Sequential([
    tf.keras.layers.Dense(24, activation=tf.nn.relu, input_shape=(window_size,)),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [19]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
size_histories['fcnn/tiny'] = compile_and_fit(tiny, train_dataset, 
                                              val_dataset,
                                              "fcnn/tiny", 
                                              optimizer=get_optimizer(), 
                                              max_epochs=num_epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                19704     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 19,729
Trainable params: 19,729
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10000
13222/13222 [==============================] - 132s 6ms/step - loss: 1.5626 - accuracy: 0.5328 - val_loss: 0.8834 - val_accuracy: 0.5184

Epoch: 0, accuracy:0.5328,  loss:1.5626,  val_accuracy:0.5184,  val_loss:0.8834,  
.Epoch 2/10000
13222/13222 [==============================] - 15s 1ms/step - loss: 0.8546 - accuracy: 0.5239 - val_loss: 0.8003 - val_accuracy: 0.5085
.Epoch 3/10000
13222/13222 [==============================] - 16s 1ms/step - loss: 0.7873 - accuracy: 0.5185 - val_loss: 0.7706 

13222/13222 [==============================] - 14s 1ms/step - loss: 0.6971 - accuracy: 0.5015 - val_loss: 0.6949 - val_accuracy: 0.5017
.Epoch 104/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6971 - accuracy: 0.5015 - val_loss: 0.6949 - val_accuracy: 0.5017
.Epoch 105/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6970 - accuracy: 0.5015 - val_loss: 0.6948 - val_accuracy: 0.5018
.Epoch 106/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6969 - accuracy: 0.5015 - val_loss: 0.6948 - val_accuracy: 0.5018
.Epoch 107/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6969 - accuracy: 0.5016 - val_loss: 0.6947 - val_accuracy: 0.5019
.Epoch 108/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6968 - accuracy: 0.5017 - val_loss: 0.6947 - val_accuracy: 0.5018
.Epoch 109/10000
13222/13222 [==============================] - 14s 1ms/step - loss: 0.6968 - accur

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(size_histories['fcnn/tiny'].history["val_accuracy"]))

In [ ]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(window_size,)),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
size_histories['fcnn/small'] = compile_and_fit(small_model, train_dataset, 
                                              test_dataset,'fcnn/small',optimizer=get_optimizer())

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(size_histories['fcnn/small'].history["val_accuracy"]))

In [ ]:
large_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(window_size,)),  
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
size_histories['fcnn/large'] = compile_and_fit(large_model, train_dataset, 
                                              test_dataset, "fcnn/large",optimizer=get_optimizer())

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(size_histories['fcnn/large'].history["val_accuracy"]))

In [ ]:
plotter_loss = tfdocs.plots.HistoryPlotter(metric = 'loss', smoothing_std=10)
plotter_loss.plot(size_histories)
plt.ylim([0., 1.5])

In [ ]:
plotter_acc = tfdocs.plots.HistoryPlotter(metric = 'accuracy', smoothing_std=10)
plotter_acc.plot(size_histories)
plt.ylim([0., 1.1])

In [ ]:
plotter_acc.plot(size_histories)
a = plt.xscale('log')
plt.xlim([0.1, max(plt.xlim())])
plt.ylim([0., 1.])
plt.xlabel("Epochs [Log Scale]")

In [ ]:
plotter_loss.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.0, 1.])
plt.xlabel("Epochs [Log Scale]")

### Dropout

In [ ]:
tiny_dropout = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(FEATURES,)),  
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
size_histories['fcnn/tiny/dropout'] = compile_and_fit(tiny_dropout,  train_dataset, 
                                                      test_dataset,"fcnn/tiny/dropout", 
                                                      optimizer=None, 
                                                      max_epochs=1000)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(size_histories['fcnn/tiny/dropout'].history["val_accuracy"]))

In [ ]:
plotter_loss = tfdocs.plots.HistoryPlotter(metric = 'loss', smoothing_std=10)
plotter_loss.plot(size_histories)
plt.ylim([0., 1.5])

In [ ]:
plotter_acc = tfdocs.plots.HistoryPlotter(metric = 'accuracy', smoothing_std=10)
plotter_acc.plot(size_histories)
plt.ylim([0., 1.1])

Since the initial model was not overfitting dropout effect is far away from being something positive for training this network. Let's analyse kernel regularization effect

### Kernel regularizer

In [ ]:
# shutil.rmtree(logdir/'regularizers/tiny', ignore_errors=True)
# shutil.copytree(logdir/'fcnn/tiny', logdir/'regularizers/tiny')
regularizer_histories = {}
regularizer_histories['large'] = size_histories['fcnn/large']
regularizer_histories['dropout'] = size_histories['fcnn/tiny/dropout']

A "simple model" in this context is a model where the distribution of parameter values has less entropy (or a model with fewer parameters altogether, as we saw in the section above). Thus a common way to mitigate overfitting is to put constraints on the complexity of a network by forcing its weights only to take small values, which makes the distribution of weight values more "regular". This is called "weight regularization", and it is done by adding to the loss function of the network a cost associated with having large weights. This cost comes in two flavors:

L1 regularization, where the cost added is proportional to the absolute value of the weights coefficients (i.e. to what is called the "L1 norm" of the weights).

L2 regularization, where the cost added is proportional to the square of the value of the weights coefficients (i.e. to what is called the squared "L2 norm" of the weights). L2 regularization is also called weight decay in the context of neural networks. Don't let the different name confuse you: weight decay is mathematically the exact same as L2 regularization.

L1 regularization pushes weights towards exactly zero encouraging a sparse model. L2 regularization will penalize the weights parameters without making them sparse since the penalty goes to zero for small weights. one reason why L2 is more common.

In tf.keras, weight regularization is added by passing weight regularizer instances to layers as keyword arguments. Let's add L2 weight regularization now.

In [ ]:
l2_model = tf.keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2),
                 input_shape=(FEATURES,)),
    layers.Dense(64, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
regularizer_histories['l2'] = compile_and_fit(l2_model, train_dataset, 
                                                      test_dataset, "regularizers/l2")

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(regularizer_histories['l2'].history["val_accuracy"]))

In [ ]:
plotter_loss.plot(regularizer_histories)
plt.ylim([0., 0.9])

In [ ]:
plotter_acc.plot(regularizer_histories)
plt.ylim([0.1, 1.])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
combined_model = tf.keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2),
                 input_shape=(FEATURES,)),
    layers.Dense(64, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dropout(dr),
    layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
regularizer_histories['combined'] = compile_and_fit(combined_model, train_dataset, 
                                                      test_dataset, "regularizers/combined")

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(regularizer_histories['combined'].history["val_accuracy"]))

In [ ]:
plotter_loss.plot(regularizer_histories)
plt.ylim([0., 0.9])
plt.xlim([5, max(plt.xlim())])

In [ ]:
plotter_acc.plot(regularizer_histories)
plt.ylim([0.4, 1.2])

## LSTM

In [ ]:
lstm_histories = {}
lstm_histories['large'] = size_histories['fcnn/large']
lstm_histories['dropout'] = size_histories['fcnn/tiny/dropout']

In [ ]:
dr=0.2
lr2=1e-8
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
lstm_large_model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=1),# expand the dimension form (50, 4096) to (50, 4096, 1)
                      input_shape=[FEATURES,]),
    tf.keras.layers.LSTM(512, activation=tf.nn.tanh, return_sequences=True,
                  kernel_regularizer=regularizers.l2(lr2)),
#     layers.Dropout(dr),
    tf.keras.layers.LSTM(512, activation=tf.nn.tanh,
                 kernel_regularizer=regularizers.l2(lr2)),
#     layers.Dropout(dr),
#     tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                  kernel_regularizer=regularizers.l2(lr2)),  
#     layers.Dropout(dr),
#     tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                  kernel_regularizer=regularizers.l2(lr2)),
#     layers.Dropout(dr),
    tf.keras.layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
#     layers.Dropout(dr),
    tf.keras.layers.Dense(24, activation=tf.nn.relu,
                 kernel_regularizer=regularizers.l2(lr2)),
    layers.Dropout(dr),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
lstm_histories['lstm/large'] = compile_and_fit(lstm_large_model, train_dataset, 
                                                      test_dataset, "lstm/large")

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('test acc:', max(lstm_histories['lstm/large'].history["val_accuracy"]))

In [ ]:
plotter_loss.plot(lstm_histories, "loss")
plt.ylim([0., 0.9])
plt.xlim([5, max(plt.xlim())])

In [ ]:
plotter_acc.plot(lstm_histories)
plt.ylim([0., 1.])
plt.xlim([5, max(plt.xlim())])